<a href="https://colab.research.google.com/github/anandhc6/Assignment-3/blob/main/Best_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import tensorflow as tf

import matplotlib.pyplot as plt
import math
import matplotlib.ticker as ticker
import numpy as np
from random import randrange 
#from google.colab import files
import pandas as pd
import random
from tensorflow import keras
#from keras import backend
#from tensorflow.python.keras import layers

#from tensorflow.python.keras.models import load_model
#from tensorflow.python.keras.callbacks import EarlyStopping
import os
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

In [2]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf 'dakshina_dataset_v1.0.tar'

--2022-05-06 12:27:10--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.128, 142.251.10.128, 142.251.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  69.9MB/s    in 34s     

2022-05-06 12:27:44 (57.1 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [3]:
batch_size = 64  # Batch size for training.
#epochs = 10  # Number of epochs to train for.
#latent_dim = 256  # Latent dimensionality of the encoding space. #hidden states hyperparameter
# Path to the data txt file on disk.
train_data = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.train.tsv"
val_data = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.dev.tsv"
# open and save the files to lists
with open(train_data, "r", encoding="utf-8") as f:
    train_lines = f.read().split("\n")
with open(val_data, "r", encoding="utf-8") as f:
    val_lines = f.read().split("\n")
# popping the last element of all the lists since it is empty character
train_lines.pop()
val_lines.pop()
random.shuffle(train_lines)
print(train_lines[0:2])


['சிற்பங்கள்\tchirpangal\t1', 'குறைபாடுகளை\tkuraipaadukalai\t1']


In [4]:
# embedding train model
def embedding_train(train_lines):

    input_texts = []
    target_texts = []
    input_characters = set()
    target_characters = set()
    # go through the train lines and split them into 3 and save input and target
    for line in train_lines[: (len(train_lines) - 1)]:
        # because we want english to devanagiri conversion
        target_text, input_text, _ = line.split("\t")
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = "\t" + target_text + "\n"
        # append it to the main input texts list
        input_texts.append(input_text)
        # append it to the main target texts list
        target_texts.append(target_text)
        # to find the number of unique characters in both
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
    # add the space character to both
    input_characters.add(" ")
    target_characters.add(" ")
    # sort it
    input_characters = sorted(list(input_characters))
    target_characters = sorted(list(target_characters))
    # find the number
    num_encoder_tokens = len(input_characters)
    num_decoder_tokens = len(target_characters)
    # find the maximum length of input word and target word
    max_encoder_seq_length = max([len(txt) for txt in input_texts])
    max_decoder_seq_length = max([len(txt) for txt in target_texts])
    
    print("Number of samples:", len(input_texts))
    print("Number of unique input tokens:", num_encoder_tokens)
    print("Number of unique output tokens:", num_decoder_tokens)
    print("Max sequence length for inputs:", max_encoder_seq_length)
    print("Max sequence length for outputs:", max_decoder_seq_length)
    # create an index
    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
   
    # create an 0 array for encoder input size of (input_texts,max_seqlen,tokens)
    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length), dtype="float32")
    # create decoder input
    decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length), dtype="float32")
    # create decoder target
    decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
    # for each sample convert it into character encoding i.e. if
    # at that position a character is present then encode the index of that character there
    # this is done for both encoder and decoder input data for further word embedding
    # but target data is one hot encoded.
    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t] = input_token_index[char]
        # remaining positions set as empty space
        encoder_input_data[i, t + 1:] = input_token_index[" "]
        # similarly do for decoder data
        for t, char in enumerate(target_text):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t] = target_token_index[char]
            # check if t >0 since decoder targer data is ahead
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
        # append both the remaining positions of both the datas with empty space
        decoder_input_data[i, t + 1:] = target_token_index[" "]
        decoder_target_data[i, t:, target_token_index[" "]] = 1.0

    return encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_tokens,num_decoder_tokens,input_token_index,target_token_index,max_encoder_seq_length,max_decoder_seq_length


In [5]:
# embedding validation data
# for validation data, almost same
def embedding_val(val_lines,num_decoder_tokens,input_token_index,target_token_index):
    val_input_texts = []
    val_target_texts = []
    
    for line in val_lines[: (len(val_lines) - 1)]:
        target_text, input_text, _ = line.split("\t")
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = "\t" + target_text + "\n"
        val_input_texts.append(input_text)
        val_target_texts.append(target_text)
    val_max_encoder_seq_length = max([len(txt) for txt in val_input_texts])
    val_max_decoder_seq_length = max([len(txt) for txt in val_target_texts])
    val_encoder_input_data = np.zeros(
        (len(val_input_texts), val_max_encoder_seq_length), dtype="float32")
    val_decoder_input_data = np.zeros(
        (len(val_input_texts), val_max_decoder_seq_length), dtype="float32")
    val_decoder_target_data = np.zeros(
        (len(val_input_texts), val_max_decoder_seq_length, num_decoder_tokens), dtype="float32")
    for i, (input_text, target_text) in enumerate(zip(val_input_texts, val_target_texts)):
        for t, char in enumerate(input_text):
            val_encoder_input_data[i, t] = input_token_index[char]
        val_encoder_input_data[i, t + 1:] = input_token_index[" "]
        for t, char in enumerate(target_text):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            val_decoder_input_data[i, t] = target_token_index[char]
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
        val_decoder_input_data[i, t + 1:] = target_token_index[" "]
        val_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

    return val_encoder_input_data,val_decoder_input_data,val_decoder_target_data,target_token_index,val_target_texts


In [6]:
#Embedding data
encoder_input_data,decoder_input_data,decoder_target_data,num_encoder_tokens,num_decoder_tokens,input_token_index,target_token_index,max_encoder_seq_length,max_decoder_seq_length = embedding_train(train_lines)
val_encoder_input_data,val_decoder_input_data,val_decoder_target_data,target_token_index,val_target_texts = embedding_val(val_lines,num_decoder_tokens,input_token_index,target_token_index)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
#reverse_target_char_index[0] = "\n"

Number of samples: 68217
Number of unique input tokens: 27
Number of unique output tokens: 49
Max sequence length for inputs: 30
Max sequence length for outputs: 28


In [7]:
import tensorflow as tf

class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
    
    Credits to Tensorflow.org and https://github.com/thushv89/attention_keras/blob/master/src/layers/attention.py
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.        

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)
            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            #print("Started context")
            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]
        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim
        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )
        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )
        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [8]:
import tensorflow as tf
def seq2seq(embedding_size, n_encoder_tokens, n_decoder_tokens, n_encoder_layers,
                 n_decoder_layers, latent_dimension, cell_type,
                 target_token_index, max_decoder_seq_length, reverse_target_char_index,
                 dropout,encoder_input_data, decoder_input_data,
                decoder_target_data,
                batch_size,epochs
):
  encoder_inputs = keras.Input(shape=(None,), name='encoder_input')
  # word embedding layer
  encoder = None
  encoder_outputs = None
  state_h = None
  state_c = None
  e_layer=n_encoder_layers

  if cell_type=="RNN":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,
                                             name='encoder_embedding')(encoder_inputs)

    encoder = keras.layers.SimpleRNN(latent_dimension, return_state=True, return_sequences=True,
                                             name='encoder_hidden_1', dropout=dropout)
    encoder_outputs, state_h = encoder(embed)

    encoder_states = None
    encoder_states = [state_h]

    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(
        decoder_inputs)
    
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.SimpleRNN(latent_dimension, return_sequences=True, return_state=True,
                                             name='decoder_hidden_1', dropout=dropout)
        
    # all decoders the initial state is encoder last state of last layer
    decoder_outputs, _ = decoder(embed_dec, initial_state=encoder_states)
    
    attn_out, attn_states = AttentionLayer(name='attention_layer')([encoder_outputs, decoder_outputs])
    decoder_concat_input = tf.keras.layers.Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])
    # hidden = keras.layers.Dense(embedding_size, activation="relu")
    # hidden_outputs = hidden(decoder_concat_input)
    #decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax",name="dense_1")
    decoder_dense = tf.keras.layers.TimeDistributed(keras.layers.Dense(n_decoder_tokens, activation="softmax",name="dense_1"))
    decoder_outputs = decoder_dense(decoder_concat_input)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(
          optimizer="rmsprop", loss="categorical_crossentropy",
          metrics=['accuracy'])#, metrics=[my_metric]                 

    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          #callbacks=WandbCallback()
      )
    model.summary()
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc = model.get_layer(
              'encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [encoder_outputs, state_h_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  # input_2
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []
    decoder_hidden_state = keras.Input(shape=(None,latent_dimension), name = "input_4")

    for j in range(1, n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs, state_h_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec]
        decoder_states_inputs += current_states_inputs


    attention_layer = model.get_layer('attention_layer')
    #decoder_outputs_att = decoder_ouputs
    attention_out, attention_states = attention_layer([decoder_hidden_state, decoder_outputs])

    decoder_concat_inf = keras.layers.Concatenate(axis=-1, name='concat_layer_inf')([decoder_outputs, attention_out])
    decoder_dense = model.get_layer("time_distributed_2")
    decoder_dense_outputs = decoder_dense(decoder_concat_inf)
    # decoder_model = keras.Model(
    #     [encoder_inputs, decoder_inputs] + decoder_states_inputs,[attention_states, decoder_dense_outputs] + decoder_states
    # )
    decoder_model=keras.Model([decoder_inputs]+[decoder_hidden_state,decoder_state_input_h],[decoder_dense_outputs]+decoder_states+[attention_states])
    return encoder_model, decoder_model

  elif cell_type=="GRU":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,
                                             name='encoder_embedding')(encoder_inputs)
    encoder = keras.layers.GRU(latent_dimension, return_state=True, return_sequences=True,
                                             name='encoder_hidden_1', dropout=dropout)
    encoder_outputs, state_h = encoder(embed)
   
    encoder_states = None
    encoder_states = [state_h]

    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(
        decoder_inputs)
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.GRU(latent_dimension, return_sequences=True, return_state=True,
                                             name='decoder_hidden_1', dropout=dropout)
    # all decoders the initial state is encoder last state of last layer
    decoder_outputs, _ = decoder(embed_dec, initial_state=encoder_states)
    attn_out, attn_states = AttentionLayer(name='attention_layer')([encoder_outputs, decoder_outputs])
    decoder_concat_input = tf.keras.layers.Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])
    decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax", name='dense_1')
    #decoder_dense = tf.keras.layers.TimeDistributed(keras.layers.Dense(n_decoder_tokens, activation="softmax",name="dense_1"))
    decoder_outputs = decoder_dense(decoder_concat_input)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(
          optimizer="rmsprop", loss="categorical_crossentropy",
          metrics=['accuracy'])#, metrics=[my_metric]                 

    # earlystopping = EarlyStopping(
    #     monitor="val_loss", min_delta=0.01, patience=5, verbose=2, mode="min")
    
    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          #validation_data=([val_encoder_input_data, val_target_texts])

          #callbacks=WandbCallback()
      )
    model.summary()
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc = model.get_layer(
              'encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [encoder_outputs,state_h_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  # input_2
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []
    decoder_hidden_state = keras.Input(shape=(None,latent_dimension), name = "input_4")


    for j in range(1, n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs, state_h_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec]
        decoder_states_inputs += current_states_inputs

    attention_layer = model.get_layer('attention_layer')
    #decoder_outputs_att = decoder_ouputs
    attention_out, attention_states = attention_layer([decoder_hidden_state, decoder_outputs])
    decoder_concat_inf = keras.layers.Concatenate(axis=-1, name='concat_layer_inf')([decoder_outputs, attention_out])

    decoder_dense = model.get_layer('dense_1')
    decoder_dense_outputs = decoder_dense(decoder_concat_inf)
    decoder_model=keras.Model([decoder_inputs]+[decoder_hidden_state,decoder_state_input_h],[decoder_dense_outputs]+decoder_states+[attention_states])

    return encoder_model, decoder_model


  elif cell_type=="LSTM":
    embed = tf.keras.layers.Embedding(input_dim=n_encoder_tokens, output_dim=embedding_size,
                                             name='encoder_embedding')(encoder_inputs)
    encoder = keras.layers.LSTM(latent_dimension, return_state=True, return_sequences=True,
                                             name='encoder_hidden_1', dropout=dropout)
    encoder_outputs, state_h, state_c = encoder(embed)
  
    encoder_states = None
    encoder_states = [state_h, state_c]

    decoder_inputs = keras.Input(shape=(None,), name='decoder_input')
    embed_dec = tf.keras.layers.Embedding(n_decoder_tokens, embedding_size, name='decoder_embedding')(
        decoder_inputs)
    # number of decoder layers
    d_layer = n_decoder_layers
    decoder = None
    decoder = keras.layers.LSTM(latent_dimension, return_sequences=True, return_state=True,
                                             name='decoder_hidden_1', dropout=dropout)
    # all decoders the initial state is encoder last state of last layer
    decoder_outputs, _,_ = decoder(embed_dec, initial_state=encoder_states)
    attn_out, attn_states = AttentionLayer(name='attention_layer')([encoder_outputs, decoder_outputs])
    decoder_concat_input = tf.keras.layers.Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])
    decoder_dense = keras.layers.Dense(n_decoder_tokens, activation="softmax", name='dense_1')
    decoder_outputs = decoder_dense(decoder_concat_input)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.compile(
          optimizer="rmsprop", loss="categorical_crossentropy",
          metrics=['accuracy'])#, metrics=[my_metric]                 
    
    # earlystopping = EarlyStopping(
    #     monitor="val_loss", min_delta=0.01, patience=5, verbose=2, mode="min")
    
    model.fit(
          [encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          #validation_data=[val_encoder_input_data, val_target_texts]
          #callbacks=WandbCallback()
      )
    model.summary()
    encoder_inputs = model.input[0]
    encoder_outputs, state_h_enc, state_c_enc = model.get_layer(
              'encoder_hidden_' + str(n_encoder_layers)).output
    encoder_states = [encoder_outputs,state_h_enc, state_c_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  # input_2
    decoder_outputs = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_states_inputs = []
    decoder_states = []
    decoder_hidden_state = keras.Input(shape=(None,latent_dimension), name = "input_4")

    for j in range(1,n_decoder_layers + 1):
        decoder_state_input_h = keras.Input(shape=(latent_dimension,))
        decoder_state_input_c = keras.Input(shape=(latent_dimension,))
        current_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder = model.get_layer('decoder_hidden_' + str(j))
        decoder_outputs_inf, state_h_dec, state_c_dec = decoder(decoder_outputs, initial_state=current_states_inputs)
        decoder_states += [state_h_dec, state_c_dec]
        decoder_states_inputs += current_states_inputs

    attention_layer = model.get_layer('attention_layer')
    #decoder_outputs_att = decoder_ouputs
    attention_out, attention_states = attention_layer([decoder_hidden_state, decoder_outputs_inf])
    decoder_concat_inf = keras.layers.Concatenate(axis=-1, name='concat_layer_inf')([decoder_outputs_inf, attention_out])

    decoder_dense = model.get_layer('dense_1')
    decoder_dense_outputs = decoder_dense(decoder_concat_inf)
    # decoder_model = keras.Model(
    #     [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
    # )
    decoder_model=keras.Model([decoder_inputs]+[decoder_hidden_state,decoder_state_input_h,decoder_state_input_c],[decoder_dense_outputs]+decoder_states+[attention_states])

    return encoder_model, decoder_model
      

In [50]:
# def decode_sequence(input_seq,n_decoder_layers,cell_type,encoder_model,decoder_model,latent_dimension):
#             # Encode the input as state vectors.
#             if cell_type=='LSTM':
#               encoder_first_outputs,state_h,state_c= encoder_model.predict(input_seq)
#               states_value=[state_h,state_c]
#             else:
#               encoder_first_outputs,states_value= encoder_model.predict(input_seq)

#             # Generate empty target sequence of length 1.
#             empty_seq = np.zeros((1, 1))
#             empty_seq[0, 0] = target_token_index["\t"]
#             target_seq = empty_seq

#             # Sampling loop for a batch of sequences
#             # (to simplify, here we assume a batch of size 1).
#             stop_condition = False
#             decoded_sentence = ""
#             attention_weights = []
#             while not stop_condition:
#                 if cell_type == "LSTM":
#                     output_tokens, h, c,attn = decoder_model.predict([target_seq, encoder_first_outputs] + states_value)
#                 elif cell_type == "RNN" or cell_type == "GRU":
#                     states_value = states_value[0].reshape((1, latent_dimension))
#                     output_tokens, h ,attn= decoder_model.predict([target_seq] + [encoder_first_outputs] + [states_value])
#                 #dec_ind = np.argmax(output_tokens, axis=-1)[0, 0]
                
#                 attention_weights.append(attn)
#                 #print(attention_weights)
#                 # print("1")
#                 # attention=np.array(attention_weights)
#                 # print(attention.reshape(attention.shape[0],attention.shape[-1]))
    
#                 # Sample a token
#                 sampled_token_index = np.argmax(output_tokens[0, -1, :])
#                 sampled_char = reverse_target_char_index[sampled_token_index]
#                 decoded_sentence += sampled_char

#                 # Exit condition: either hit max length
#                 # or find stop character.
#                 if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
#                     stop_condition = True

#                 # Update the target sequence (of length 1).
#                 target_seq = np.zeros((1, 1))
#                 target_seq[0, 0]= sampled_token_index

#                 # Update states
#                 if cell_type == "LSTM":
#                     states_value = [h, c]
#                 elif cell_type == "RNN" or cell_type == "GRU":
#                     states_value = [h]
#             return decoded_sentence,attention_weights

In [66]:
import numpy as np
from IPython.display import HTML
from IPython.display import display
import ast

def softmax(x):
    denom = sum([np.exp(p) for p in x])
    return [np.exp(p) / denom for p in x]

def cstr(s, color = 'black'):
    return "<text style=color:#000;padding-top:1.5px;padding-bottom:1.5px;padding-left:2.5px;padding-right:2.5px;background-color:{}>{} </text>".format(color, s)

def get_clr(value, mode):
    if(mode == 'l'):
        colors = ['#85c2e1', '#89c4e2', '#95cae5', '#99cce6', '#a1d0e8', '#b2d9ec', '#baddee', '#c2e1f0', '#eff7fb', '#f9e8e8', '#f9e8e8', '#f9d4d4', '#f9bdbd', '#f8a8a8', '#f68f8f', '#f47676', '#f45f5f', '#f34343', '#f33b3b', '#f42e2e']
        value = int((value * 100) / 5)
        return colors[value]
    else:
        # colors = ['#FFFFFF','#DFFFFF','#BFFFFF','#9FFFFF','#7FFFFF','#5FFFFF','#3FFFFF','#03FFFF','#00EFFF','#00DFFF','#00CFFF','#00BFFF','#00AFFF','#009FFF']
        # factor = 0.07142857142857142
        # color_index = int(value/factor)
        # return colors[color_index]
        colors = ['#ffffff', '#ecf7fb', '#daeff7', '#c7e7f3', '#b5dfef', '#a2d7eb', '#90cfe7', '#7dc7e3', '#6abfdf', '#58b7db', '#46afd7']
        value = int((value * 100) / 10)
        return colors[value]

def visualize_c(dec_char, text_colours):
    if (dec_char == "<e>"):
      display(HTML(''.join([cstr(ti, color = ci) for ti, ci in text_colours]) + " <b> &emsp; &lt; e &gt; </b>  &emsp; &nbsp; "))
    else:
      display(HTML(''.join([cstr(ti, color = ci) for ti, ci in text_colours]) + " <b> &emsp; {}</b>  &emsp; &emsp; ".format(dec_char)))

def visualize_l(dec_seq, prob):
    text_colours = []

    for c, p in zip(dec_seq, prob): 
        text = (c, get_clr(p, 'l'))
        text_colours.append(text)
    
    display(HTML(''.join([cstr(ti, color = ci) for ti, ci in text_colours])))

def visualize_connectivity(N):

    # Reading from conv_vis file
    with open("conn_vis.txt", "r", encoding='utf-8') as filepointer:
        
        lines = filepointer.readlines()
        #print("lines",lines)
        i = 0
        words_visualized = 0

        while i < len(lines) and  words_visualized< N:
            line = lines[i]
            #print("line:",line)
            if line[:4] == "Next":
                words_visualized += 1
                i += 1
                continue

            if line[:4] != "Next": 
                true_word, dec_char_len = line.split('\t') 
                #print(len(true_word))
                dec_word_len = int(dec_char_len)
                i += 1

                true_word_array = [c for c in true_word]
                #print(true_word_array)

                for j in range(dec_word_len):
                    line = lines[i]
                    #print(line)                   
                    line = line.split('\t')
                    #print(line)
  
                    dec_char = line[0]
                    text_colours = []

                    prob = []
                    for prob_index in range(1,len(true_word)+1) :
                        print(prob_index)
                        p = float(line[prob_index])
                        prob.append(p)

                    line = softmax(prob)

                    
                    for prob_index in range(len(true_word)) :
                        p = float(line[prob_index])

                        true_char = true_word_array[prob_index]
                        text= (true_char, get_clr(p, 'c') )
                        text_colours.append(text)

                    visualize_c(dec_char, text_colours)
            
                    i += 1

            print("\n\n")

def visualize_lstm(N, neuron):

    for i in range(N):

        file = open("lstm_vis_" + str(i) + ".txt", "r")
        input_seq = file.readline()[:-1]
        
        dec_seq = []
        prob = []

        for line in file:
            temp = line.split('\t')
            dec_seq.append(temp[0])
            prob.append(ast.literal_eval(temp[1][:-1])[neuron - 1])

        visualize_l(dec_seq, prob)
        print()



In [63]:
def decode_conn(input_seq):
            # Encode the input as state vectors.
            if cell_type=='LSTM':
              encoder_first_outputs,state_h,state_c= encoder_model.predict(input_seq)
              states_value=[state_h,state_c]
            else:
              encoder_first_outputs,states_value= encoder_model.predict(input_seq)

            # Generate empty target sequence of length 1.
            empty_seq = np.zeros((1, 1))
            empty_seq[0, 0] = target_token_index["\t"]
            target_seq = empty_seq

            # Sampling loop for a batch of sequences
            # (to simplify, here we assume a batch of size 1).
            stop_condition = False
            decoded_sentence = ""
            attention_weights = []
            visualization_data = []
            heatmap_data = []
            while not stop_condition:
                if cell_type == "LSTM":
                    output_tokens, h, c,attn = decoder_model.predict([target_seq, encoder_first_outputs] + states_value)
                elif cell_type == "RNN" or cell_type == "GRU":
                    states_value = states_value[0].reshape((1, latent_dimension))
                    output_tokens, h ,attn= decoder_model.predict([target_seq] + [encoder_first_outputs] + [states_value])
                dec_ind = np.argmax(output_tokens, axis=-1)[0, 0]
                #attention_weights.append((dec_ind, attn))
                # Sample a token
                sampled_token_index = np.argmax(output_tokens[0, -1, :])
                sampled_char = reverse_target_char_index[sampled_token_index]
                decoded_sentence += sampled_char

                # Exit condition: either hit max length
                # or find stop character.
                if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
                    stop_condition = True

                # Update the target sequence (of length 1).
                target_seq = np.zeros((1, 1))
                target_seq[0, 0]= sampled_token_index
                visualization_data.append((sampled_char, states_value[0]))
                heatmap_data.append((sampled_char, attn))
                # Update states
                if cell_type == "LSTM":
                    states_value = [h, c]
                elif cell_type == "RNN" or cell_type == "GRU":
                    states_value = [h]
            return decoded_sentence,heatmap_data ,visualization_data

In [64]:
def sigmoid(x):
    return [1/(1 + np.exp(-z)) for z in x]

In [73]:
from random import sample
for_test=True
subset=10
count, visual_count, test_size = 0, 0, len(test_input_texts[0:subset])
predictions_attention = open("predictions_attention.csv", "w", encoding='utf-8')
predictions_attention.write("Input Sentence,Predicted Sentence,Original Sentence\n")  

visualisation_inputs = sample(range(test_size), 10)
#visualisation_inputs = range(0,2)
heatmaps = []


for seq_index in range(test_size):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = test_encoder_input_data[seq_index : seq_index + 1]
    decoded_word, heatmap_data, visualization_data = decode_conn(input_seq)
    orig_word = test_target_texts[seq_index][1:]

    # print("-")
    # print("Input sentence:", test_input_words[seq_index])
    # print("Decoded sentence:", decoded_word[:-1])
    # print("Original sentence:", orig_word[:-1])

    predictions_attention.write(test_input_texts[seq_index] + "," + decoded_word[:-1] + "," + orig_word[:-1] + "\n")

    if(orig_word == decoded_word): count += 1
    print(orig_word)
    print(decoded_word)
    
    if for_test:
      if seq_index in visualisation_inputs:
          
          # # Heatmap Pl
          # heatmap = attention_heatmap(test_input_texts[seq_index], heatmap_data)
          # #plt.show(heatmap)
          # heatmaps.append(heatmap)
          
          print("create conn.txt")
          # Connectivity Visualization
          with open("conn_vis.txt", "a", encoding='utf-8') as filepointer:

              '''' The logic to compute the  heatmap and true word '''

              true_word = test_input_texts[seq_index]

              ''' Writing data into the conv_vis.txt file for visualisation purpose '''
              
              filepointer.write(true_word)
              filepointer.write("\t")
              filepointer.write(str(len(heatmap_data)))
              filepointer.write("\n")

              for tup in range(len(heatmap_data)):
                  dec_char = heatmap_data[tup][0]
                  dec_char_prob = heatmap_data[tup][1].reshape(-1)
                  #print(dec_char)
                  #print(dec_char_prob)
                  if tup == len(heatmap_data) - 1:
                      filepointer.write("<e>")
                  else:
                      filepointer.write(dec_char)
                
                  filepointer.write("\t")

                  for p in range(len(true_word)):
                      filepointer.write(str(dec_char_prob[p]))
                      filepointer.write("\t")

                  filepointer.write("\n")

              filepointer.write("Next\n")

          #print("nee")
          # LSTM Visualization
          file = open("lstm_vis_" + str(visual_count) + ".txt", "w", encoding='utf-8')
          file.write(test_input_texts[seq_index] + "\n")

          for i, data in enumerate(visualization_data):
      
              dec_char, neuron_activation  = data[0], sigmoid(data[1].reshape(-1))
              
              if i == len(visualization_data) - 1:
                  file.write("<e>" + "\t" + str(neuron_activation) + "\n")
              else:
                  file.write(dec_char + "\t" + str(neuron_activation) + "\n")

          visual_count += 1

ஃபார்ம்

பார்ம்

create conn.txt
ஃபார்ம்

பரம்

create conn.txt
ஃபார்ம்

போர்ம்

create conn.txt
ஃபார்ம்

பார்ம்

create conn.txt
ஃபேஸ்

பேக்ஸ்

create conn.txt
ஃபேஸ்

ஃபேஸ்

create conn.txt
ஃபேஸ்

பேஸ்

create conn.txt
ஃபேஸ்

பாஸ்

create conn.txt
ஃபேஸ்

பாஷ்

create conn.txt
அஇஅதிமுக

ஏதிமுக

create conn.txt


In [69]:
visualize_connectivity(10)
visualize_lstm(10, 0)

In [61]:
def accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts ,verbose=True):
        n_correct = 0
        n_total = 0
        index=1
        queries = []
        outputs = []
        ground_truth=[]
        attentions = []
        fig = plt.figure(figsize=(10,10))
        for seq_index in range(len(val_encoder_input_data)):
            # Take one sequence (part of the training set)
            # for trying out decoding.
            input_seq = val_encoder_input_data[seq_index: seq_index + 1]
            # Generate empty target sequence of length 1.
            # empty_seq = np.zeros((1, 1))
            # # Populate the first character of target sequence with the start character.
            # empty_seq[0, 0] =  target_token_index["\t"]
            decoded_sentence,att = decode_sequence(input_seq,n_decoder_layers,cell_type,encoder_model,decoder_model,latent_dimension)
            #print(att)
            # print("Done 1")
            if decoded_sentence.strip() == val_target_texts[seq_index].strip():
                n_correct += 1

            n_total += 1
            #print(n_correct)

            if verbose:
                print('Prediction ', decoded_sentence.strip(), ',Ground Truth ', val_target_texts[seq_index].strip())
            
            #print(att)
            attention=np.array(att)
            print(input_seq)
            print(attention.reshape(attention.shape[0],attention.shape[-1])[:len(decoded_sentence),:])
            print("1")
            print(attention.shape)
            print(attention.shape[-1])
            # plt.subplot(3,3,index)
            # plt.imshow(attention.reshape(attention.shape[0],attention.shape[-1])[:,:len(decoded_sentence)],cmap="Blues")

            # plt.xticks(range(attention.shape[0]),decoded_sentence,fontproperties=FontProperties(fname = 'Nirmala.ttf'))
            # plt.yticks(range(attention.shape[0]),test_input_texts[seq_index])

            queries.append(test_input_texts[seq_index])
            outputs.append(decoded_sentence.strip())
            #attentions.append(attention.reshape(attention.shape[0],attention.shape[-1])[:,:len(decoded_sentence)])
            ground_truth.append(val_target_texts[seq_index].strip()) # To remove the tab and newline characters
            index+=1
        #plt.show()
        df_train = pd.DataFrame({"Input": queries, "Ground Truth" : ground_truth, "Model output":outputs})
        #print(df_train)
        df_train.to_csv('predictions_attention.csv', index=False)
        return n_correct * 100.0 / n_total

In [60]:
subset = 1
test_accuracy = accuracy(test_encoder_input_data[0:subset], test_target_texts[0:subset],n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts[0:subset]) if subset>0 \
    else accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts)
print('Test accuracy: ', test_accuracy)

Prediction  பார்ம் ,Ground Truth  ஃபார்ம்
[[ 6.  1.  1. 18. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.]]
[[9.0325630e-01 5.7176527e-02 2.5399841e-04 2.1971018e-06 2.6246675e-05
  5.6690384e-05 1.1574106e-04]
 [1.2872710e-02 3.2256857e-02 8.9368850e-01 3.1814598e-02 1.7810657e-03
  6.1841944e-04 4.0256290e-04]
 [6.0860103e-04 1.0331625e-03 1.6106665e-02 9.2713672e-01 3.8119260e-02
  6.8390847e-04 1.4776373e-04]
 [1.1275454e-03 2.2427925e-04 1.7667808e-04 9.9554927e-05 9.2943138e-01
  3.3093456e-02 4.9891551e-03]
 [4.9820734e-05 6.9900113e-04 2.9182725e-04 8.0647929e-05 9.7933620e-01
  9.9417614e-03 6.6888786e-04]
 [1.7962824e-05 1.6821577e-04 3.6890045e-04 5.6519516e-06 7.4114540e-04
  1.5436269e-01 4.2091426e-01]
 [1.2164499e-05 1.3966828e-04 3.3453955e-05 8.1282758e-07 1.6524042e-04
  2.2285013e-03 3.2109812e-02]]
1
(7, 1, 1, 23)
23
Test accuracy:  0.0


<Figure size 720x720 with 0 Axes>

In [11]:
# parameters
embedding_size=128
n_encoder_tokens=num_encoder_tokens
n_decoder_tokens=num_decoder_tokens
n_encoder_layers=1
n_decoder_layers=1
latent_dimension=256
cell_type='LSTM'
target_token_index=target_token_index
max_decoder_seq_length=max_decoder_seq_length
reverse_target_char_index=reverse_target_char_index
dropout=0.2

In [12]:
# compute test accuracy
from matplotlib.font_manager import FontProperties
print('Reading test data')
test_data = "dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.test.tsv"
# open and save the files to lists
with open(test_data, "r", encoding="utf-8") as f:
    test_lines = f.read().split("\n")
# popping the last element of all the lists since it is empty character
test_lines.pop()
# embedding test
# for test data, almost same
test_input_texts = []
test_target_texts = []
for line in test_lines[: (len(test_lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    test_input_texts.append(input_text)
    test_target_texts.append(target_text)
test_max_encoder_seq_length = max([len(txt) for txt in test_input_texts])
test_max_decoder_seq_length = max([len(txt) for txt in test_target_texts])
test_encoder_input_data = np.zeros(
    (len(test_input_texts), test_max_encoder_seq_length), dtype="float32"
)
test_decoder_input_data = np.zeros(
    (len(test_input_texts), test_max_decoder_seq_length), dtype="float32"
)
test_decoder_target_data = np.zeros(
    (len(test_input_texts), test_max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
for i, (input_text, target_text) in enumerate(zip(test_input_texts, test_target_texts)):
    for t, char in enumerate(input_text):
        test_encoder_input_data[i, t] = input_token_index[char]
    test_encoder_input_data[i, t + 1:] = input_token_index[" "]
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        test_decoder_input_data[i, t] = target_token_index[char]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            test_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    test_decoder_input_data[i, t + 1:] = target_token_index[" "]
    test_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Reading test data


In [19]:
#calling rnn
# encoder_model, decoder_model=seq2seq(embedding_size, n_encoder_tokens,n_decoder_tokens,n_encoder_layers, n_decoder_layers,latent_dimension,
#                 cell_type, target_token_index, max_decoder_seq_length,reverse_target_char_index, dropout ,encoder_input_data, decoder_input_data,
#                 decoder_target_data,batch_size,10)


# val_accuracy= accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model)
# print('Validation accuracy: ', val_accuracy)

subset = 0
test_accuracy = accuracy(test_encoder_input_data[0:subset], test_target_texts[0:subset],n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts[0:subset]) if subset>0 \
    else accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension,test_input_texts)
print('Test accuracy: ', test_accuracy)

Test accuracy:  55.99179607383534


<Figure size 720x720 with 0 Axes>

In [ ]:
df = pd.DataFrame(columns=['Source', 'Predictions', 'GroundTruth'])
n_correct = 0
n_total = 0
for seq_index in range(len(test_encoder_input_data)):
    decoded_sentence = beam_search(test_encoder_input_data[seq_index:seq_index+1],encoder_model,decoder_model,beamSize,n_decoder_layers,cell_type)

    if test_target_texts[seq_index].strip() == decoded_sentence[0][0].strip():
        n_correct += 1

    n_total += 1
    row = {}
    row['SourceText'] = test_input_texts[seq_index].strip()
    row['GroundTruth'] = test_target_texts[seq_index].strip()
    row['Prediction'] = decoded_sentence[0][0].strip()
    df = df.append(row, ignore_index=True)
df.to_csv('predictions_'+str(beamSize)+'.csv', index=False)  
test_accuracy[beamSize] = (n_correct * 100.0 / n_total)
print('Test accuracy ', test_accuracy)

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

In [ ]:
def train():
  config_defaults = {
            "epochs":15,
            "embedding_size":256,
            "latent_dimension":256,
            "cell_type":'LSTM',
            "dropout":0.2  
        }
  wandb.init(config=config_defaults)
  
  config = wandb.config
  cell_type=config.cell_type
  embedding_size=config.embedding_size
  latent_dimension=config.latent_dimension
  dropout=config.dropout
  epochs=config.epochs
  run_name = "cell_type_{}_drop_{}_emd_{}_ld_{}".format(cell_type, dropout, embedding_size, latent_dimension )
  
  encoder_model, decoder_model=seq2seq(embedding_size, n_encoder_tokens,n_decoder_tokens,n_encoder_layers, n_decoder_layers,latent_dimension,
                cell_type, target_token_index, max_decoder_seq_length,reverse_target_char_index, dropout ,encoder_input_data, decoder_input_data,
                decoder_target_data,batch_size,epochs)
  
  val_accuracy=accuracy(val_encoder_input_data, val_target_texts,n_decoder_layers,encoder_model,decoder_model,cell_type,latent_dimension)
  print("Validation Accuracy:", val_accuracy)
  wandb.log({'val_accuracy': val_accuracy})
  wandb.run.name = run_name
  wandb.run.save()
  wandb.run.finish()

In [ ]:
sweep_config = {
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'val_acc',
      'goal': 'maximize'   
    },
    'early_terminate':{
  'type': 'hyperband',
  'min_iter': 5
    },
    'parameters': {
        'epochs':{
          'values':[10,15]  
        },
        #embedding
        'embedding_size': {
            'values': [128,64,256]
        },
        'latent_dimension': {
            'values': [128,256,512]
        },
        #100% means no fine tuning, only pre training
        'cell_type':{
            'values': ['RNN','GRU','LSTM']
        
        },
        'dropout': {
            'values':[0,0.2,0.3]
        },

    }
}

# sweep_id=wandb.sweep(sweep_config,entity="anandh", project = "CS6910_Assignment3_Attention")
# wandb.agent(sweep_id, train,count=5)
sweep_id="yomq8g4r"
wandb.agent(sweep_id, train, entity="anandh", project = "CS6910_Assignment3_Attention", count = 5)

wandb: Agent Starting Run: 5gm0euz0 with config:
wandb: 	cell_type: RNN
wandb: 	dropout: 0.3
wandb: 	embedding_size: 64
wandb: 	epochs: 10
wandb: 	latent_dimension: 256
